In [1]:
import os
import numpy as np
import librosa
import librosa.display
from pathlib import Path
import matplotlib.pyplot as plt
import scipy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#a function extracting on 1 feature for the model
def extract_features(signal):
    return [
        #mfcc
        librosa.feature.mfcc(y=signal,n_mfcc=20)
    ]

In [3]:
#REPLACE THE TESTING DATA PATH WITH DESIRED PATH
#training data
path_car_train = 'training/car/*.wav'
path_tram_train = 'training/tram/*.wav'
#validation data
path_car_valid = 'validation/car/*.wav'
path_tram_valid = 'validation/tram/*.wav'
#testing data
path_car_test = 'testing/car/*.wav'
path_tram_test = 'testing/tram/*.wav'

car_train = [librosa.load(os.fspath(p))[0] for p in Path().glob(path_car_train)]
tram_train = [librosa.load(os.fspath(p))[0] for p in Path().glob(path_tram_train)]
print('{} car training signals - {} tram training signals'.format(len(car_train), len(tram_train)))

car_valid = [librosa.load(os.fspath(p))[0] for p in Path().glob(path_car_valid)]
tram_valid = [librosa.load(os.fspath(p))[0] for p in Path().glob(path_tram_valid)]
print('{} car validation signals - {} tram validation signals'.format(len(car_valid), len(tram_valid)))

car_test = [librosa.load(os.fspath(p))[0] for p in Path().glob(path_car_test)]
tram_test = [librosa.load(os.fspath(p))[0] for p in Path().glob(path_tram_test)]
print('{} car testing signals - {} tram testing signals'.format(len(car_test), len(tram_test)))

103 car training signals - 96 tram training signals
18 car validation signals - 21 tram validation signals
20 car testing signals - 20 tram testing signals


In [5]:
#Preparing the data and the corresponding ground truth classes
'''
In this part, for simplicity later in the model, we will prepare the data
and its corresponding class (car/tram) (in case ground truth class is not provided)

For the corresponding class:
    - Car = "1"
    - Tram = "0"
'''
#training data
train_class1 = np.ones((len(car_train),1))
train_class2 = np.zeros((len(tram_train),1))
train_class = np.concatenate((train_class1, train_class2))
car_features_train = np.array([np.mean(extract_features(x),axis=2) for x in car_train])
tram_features_train = np.array([np.mean(extract_features(x),axis=2) for x in tram_train])
train_data = np.vstack((car_features_train, tram_features_train))
train_data[:,0] = train_data[:,0]/train_data[:,0].max() #normalizing
train_data = train_data.reshape((int(len(car_train) + len(tram_train)), 20)) #reshaping

print("Train class array size: " + str(train_class.shape))
print("Train data array size: " + str(train_data.shape))

#validation data
valid_class1 = np.ones((len(car_valid), 1))
valid_class2 = np.zeros((len(tram_valid), 1))
valid_class = np.concatenate((valid_class1, valid_class2))
car_features_valid = np.array([np.mean(extract_features(x),axis=2) for x in car_valid])
tram_features_valid = np.array([np.mean(extract_features(x),axis=2) for x in tram_valid])
valid_data = np.vstack((car_features_valid, tram_features_valid))
valid_data[:,0] = valid_data[:,0]/valid_data[:,0].max() #normalizing
valid_data = valid_data.reshape((int(len(car_valid) + len(tram_valid)), 20)) #reshaping

print("Validation vlass array size: " + str(valid_class.shape))
print("Validation data array size: " + str(valid_data.shape))

#testing data
test_class1 = np.ones((len(car_test), 1))
test_class2 = np.zeros((len(tram_test), 1))
test_class = np.concatenate((test_class1, test_class2))
car_features_test = np.array([np.mean(extract_features(x),axis=2) for x in car_test])
tram_features_test = np.array([np.mean(extract_features(x),axis=2) for x in tram_test])
test_data = np.vstack((car_features_test, tram_features_test))
test_data[:,0] =test_data[:,0]/test_data[:,0].max() #normalizing
test_data = test_data.reshape((int(len(car_test) + len(tram_test)), 20)) #reshaping

print("Testing class array size: " + str(test_class.shape))
print("Testing data array size: " + str(test_data.shape))

Train class array size: (199, 1)
Train data array size: (199, 20)
Validation vlass array size: (39, 1)
Validation data array size: (39, 20)
Testing class array size: (40, 1)
Testing data array size: (40, 20)


In [6]:
#the final model 
saved_w = 'distance'
#saved_w = 'uniform'
saved_k = 7
#saved_k = 9
def k_nearest_neighbor(X):
    '''
    X: array of input that wants to be classified
    X can have length 1 or more
    '''
    knn = KNeighborsClassifier(n_neighbors= saved_k, weights = saved_w)
    knn.fit(train_data, train_class.ravel())
    return knn.predict(X)

In [7]:
#Testing with our own testing files
test_pred = k_nearest_neighbor(test_data)
print(classification_report(test_pred, test_class))

              precision    recall  f1-score   support

         0.0       1.00      0.95      0.98        21
         1.0       0.95      1.00      0.97        19

    accuracy                           0.97        40
   macro avg       0.97      0.98      0.97        40
weighted avg       0.98      0.97      0.98        40

